In [32]:
#Tahminler gözlem benzerliğine göre yapılır
#Bana arkadaşını söyle sana kim olduğunu söyleyeyim.
#En yakın komşu metodu....
#Komşuların ortalaması işte

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

In [30]:
hit = pd.read_csv('Hitters.csv')
df=hit.copy()
df=df.dropna()
ms=pd.get_dummies(df[['League','Division','NewLeague']])
y=df['Salary']
X_=df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
X=pd.concat([X_,ms[['League_N','Division_W','NewLeague_N']]],axis=1)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)


In [5]:
knn_model=KNeighborsRegressor().fit(X_train,y_train)

In [13]:
predicted=knn_model.predict(X_test)

In [14]:
np.sqrt(mean_squared_error(predicted,y_test))

426.6570764525201

In [15]:
#Deli dehşet test hatası, tune edelim.
#Önce bi komşu sayısına göre hatalar nasıl değişiyor bakalım

In [18]:
RMSE =[]
for k in range(10):
    k=k+1
    knn_dummy_model=KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
    y_pred=knn_dummy_model.predict(X_train)
    rmse=np.sqrt(mean_squared_error(y_pred,y_train))
    RMSE.append(rmse)
    print('k = ',k,' için RMSE değeri : ', rmse)

k =  1  için RMSE değeri :  0.0
k =  2  için RMSE değeri :  179.52761335480352
k =  3  için RMSE değeri :  205.20157172291863
k =  4  için RMSE değeri :  220.5139794876305
k =  5  için RMSE değeri :  239.64671325413764
k =  6  için RMSE değeri :  243.5904190007242
k =  7  için RMSE değeri :  258.1478781634636
k =  8  için RMSE değeri :  266.05374203349805
k =  9  için RMSE değeri :  269.73782093553376
k =  10  için RMSE değeri :  271.2798300436963


In [20]:
#Cross validation --> gridsearchCV ile optimum k sayısı
#tuning başlıyor

In [22]:
#Bundan sonra GridSearchCV ile arıycaz optimum hiperparametreyi.
#hangi algoritmayı kullanıyorsak onun aranacak argümanları yazılır.

In [24]:
knn_params={'n_neighbors': np.arange(1,30,1)}

In [25]:
knn=KNeighborsRegressor()

In [26]:
knn_cv_model=GridSearchCV(knn,knn_params,cv=10)

In [27]:
knn_cv_model.fit(X_train,y_train)

C:\Users\Samet Sökel\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
#CV model en uygun komşu sayısını buldu


In [34]:
knn_cv_model.best_params_['n_neighbors']

8

In [35]:
knn_tuned=KNeighborsRegressor(n_neighbors=8).fit(X_train,y_train)

In [37]:
np.sqrt(mean_squared_error(knn_tuned.predict(X_test),y_test))

413.7094731463598